In [1]:
import pandas as pd
from nltk.util import ngrams
from nltk.stem import PorterStemmer
ps = PorterStemmer()
filename = "train.csv"

In [2]:
data = pd.read_csv(filename, header=0)
data.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
data["comment_text"] = data["comment_text"].fillna("_na_").values
#data["comment_text"]= data["comment_text"].str.replace("\n", " ")
print(data.shape)
data.head(10)
#Take 10,000 samples. 
data = data.head(1000)
print(data.shape)

(159571, 8)
(1000, 8)


In [4]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import pandas as pd
tokenizer = RegexpTokenizer(r'\w+')

In [5]:
def filter_sentences(text):
    stop_words = set(stopwords.words('english'))  
    word_tokens = tokenizer.tokenize(text)
    word_tokens = [ps.stem(w) for w in word_tokens]
    filtered_sentence = [w.lower() for w in word_tokens if not w in stop_words and not w.isdigit()]
    return " ".join(filtered_sentence)

In [6]:
list_of_comments = list(data["comment_text"])

filtered_sentences = []
for comment in list_of_comments:
    filtered_sentences.append(filter_sentences(comment))

data["comment_text"] = pd.Series(filtered_sentences)
data.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explan whi edit made usernam hardcor metallica...,0,0,0,0,0,0
1,000103f0d9cfb60f,d aww he match thi background colour i seeming...,0,0,0,0,0,0
2,000113f07ec002fd,hey man i realli tri edit war it thi guy const...,0,0,0,0,0,0
3,0001b41b1c6bb37e,i make ani real suggest improv i wonder sectio...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero ani chanc rememb page,0,0,0,0,0,0


In [7]:
unique_words = dict()
all_comments = list(data["comment_text"])
# Get the number of unique words
for comment in all_comments:
    word_tokens = tokenizer.tokenize(comment)
    for word in word_tokens:
        if word not in unique_words:
            unique_words[word] = 1

In [8]:
len(unique_words)

7088

In [9]:
all_bigrams = {}
for comment in all_comments:
    token = word_tokenize(comment)
    bigrams = list(ngrams(token,2))
    for bigram_pair in bigrams:
        if bigram_pair not in all_bigrams:
            all_bigrams[bigram_pair] = 1
    

In [10]:
len(all_bigrams)

32323

In [11]:
# Class distribution
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
for label in labels:
    count = sum(data[label]==1)
    print("There are: "+str(count)+" samples for label: "+label)

There are: 105 samples for label: toxic
There are: 9 samples for label: severe_toxic
There are: 49 samples for label: obscene
There are: 4 samples for label: threat
There are: 54 samples for label: insult
There are: 9 samples for label: identity_hate


In [12]:
# Sample code to convert list into Dataframe
col_names = ["a", "b", "c"]
data_new = [[1,1,1], [0,0,0], [1,0,1]]
df = pd.DataFrame(data_new, columns = col_names)

In [13]:
all_bigrams_list = [bigram_pair[0]+":"+bigram_pair[1] for bigram_pair in all_bigrams.keys()]

In [14]:
col_names  = all_bigrams_list + labels

In [15]:
for index in range(len(data)):
    print(list(data.iloc[index+4]))
    break
    

['0001d958c54c6e35', 'sir hero ani chanc rememb page', 0, 0, 0, 0, 0, 0]


In [16]:
final_data = []
for record_index in range(len(data)):
    new_record_for_final_data = []
    current_record = data.iloc[record_index]
    
    #Current_record is a data frame
    comment = current_record["comment_text"]
    
    token = word_tokenize(comment)
    bigrams = list(ngrams(token,2))
    all_bigrams_dict = {}
    
    for bigram_pair in bigrams:
        modified_bigram_pair = bigram_pair[0]+":"+bigram_pair[1] #Same way as in all_bigrams_list
        if modified_bigram_pair not in all_bigrams:
            all_bigrams_dict[modified_bigram_pair] = 1
        else:
            all_bigrams_dict[modified_bigram_pair] +=1
    
    # Now we have bigram count as a dictionary for this record.
    # We have to check this against all the bi-grams of the corpus stored in all_bigrams_list
    for i in range(len(all_bigrams_list)):
        if all_bigrams_list[i] not in all_bigrams_dict:
            new_record_for_final_data.append(0)
        else:
            new_record_for_final_data.append(all_bigrams_dict[all_bigrams_list[i]])
    
    
    for cur_label in labels:
        new_record_for_final_data.append(current_record[cur_label])
    
    if len(new_record_for_final_data)!=len(col_names):
        print("Error: This should not happen")
        
    else:
        final_data.append(new_record_for_final_data)
        
 


In [17]:
df = pd.DataFrame(final_data, columns = col_names)
df.to_csv("final_data_1k.csv", encoding='utf-8', index=False)